In [ ]:
import numpy as np
from utils import *
import matplotlib.pyplot as plt
import re
LAMBDA = 5

In [ ]:
def cal_flap_thickness(af):
    lh_idx = np.argmin(af[:,0])
    lh_x = af[lh_idx, 0]
    # Get trailing head
    th_x = np.minimum(af[0,0], af[-1,0])
    # Interpolate
    f_up = interp1d(af[:lh_idx+1,0], af[:lh_idx+1,1])
    f_low = interp1d(af[lh_idx:,0], af[lh_idx:,1])
    xx = np.linspace(lh_x, th_x, num=1000)
    yy_up = f_up(xx)
    yy_low = f_low(xx)
    flap = yy_up[900:] - yy_low[900:]
    return flap.mean()

points = 'results/20150114-50 +2 d.dat'
airfoil = np.loadtxt(points, skiprows=1)
print(cal_flap_thickness(airfoil))

points = 'BETTER/Airfoils1D_198_-2.0_0.7B.dat'
airfoil = np.loadtxt(points, skiprows=1)
print(cal_flap_thickness(airfoil))
show_airfoil(airfoil)
# a_up = np.concatenate([np.zeros([10]), np.array([0.0002, 0.0002, 0.0002, 0.0002, 0.00001])])
# a_low = np.concatenate([np.zeros([10]), np.array([-0.0002, -0.0002, -0.0002, -0.0002, -0.00001])])
# airfoil = mute_airfoil(airfoil, a_up, a_low)
# show_airfoil(airfoil)
# print(cal_flap_thickness(airfoil))
cd, _ = evalpreset(airfoil)
perf, _, _ = evalperf(setflap(airfoil, 2, 0.7), cl = 0.65, Re=58000)

print('perf: %.4f, cd: %.6f, thickness: %.4f %.2f, tail: %.4f, %d' % (perf, cd, cal_thickness(airfoil), cal_thickness_percent(airfoil), cal_flap_thickness(airfoil), check_backpoint(airfoil)))

# af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, Re2= 400000, lamda = LAMBDA, check_thickness=False, modify_thickness = True)
# name = points.split('/')[1].split('.')[0]+f'_{a}_{b}F'
# np.savetxt('results/_best.dat', airfoil, header='best', comments="")
# print(f'R: {R}, angle: {a}, pose: {b}, perf: {perf}, cruise cd: {cd}, launch cd: {CD_BL}')
# show_airfoil(af)
# print(f'intersection: {detect_intersect(af)}, thickness: {cal_thickness(af)}, thickness pose: {cal_thickness_percent(af)}, tail cross pose: {check_backpoint(af)}')

In [ ]:
from utils import *
import re
import os
LAMBDA = 3
dat = re.compile('.dat')
root = 'BETTER/'

airfoils = []
for path, dir, files in os.walk(root):
    for file in files:
        if dat.search(file) is not None:
            airfoils.append(file)

Rbl = 1
files.sort()
for file in airfoils:
    points = '{}{}'.format(root, file)
    airfoil = np.loadtxt(points, skiprows=1)
    af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, Re2= 400000, lamda = LAMBDA, check_thickness=False, modify_thickness = True)
    if perf > 39.06369801476684 and CD_BL < 0.00485806493088603:
        name = points.split('/')[1].split('.')[0]+f'_{a}_{b}B'
        np.savetxt('BETTER/'+name+'.dat', af, header=name, comments="")
        print(f'R: {R}, angle: {a}, pose: {b}, perf: {perf}, cruise cd: {cd}, launch cd: {CD_BL}, intersection: {detect_intersect(af)}, thickness: {cal_thickness(af)}, thickness pose: {cal_thickness_percent(af)}, tail cross pose: {check_backpoint(af)}')
    os.remove(points)
    print('{} removed'.format(points))

In [ ]:
points = 'samples/DiffusionAirfoil1DTransform_006.dat'
airfoil = np.loadtxt(points, skiprows=1)
a_up = np.concatenate([np.zeros([10]), np.ones([4]) * 0.001, np.array([0.0001])])
a_low = np.concatenate([np.zeros([10]), np.ones([4]) * -0.001, np.array([0.0001])])
airfoil = mute_airfoil(airfoil, a_up, a_low)
show_airfoil(airfoil)
af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, Re2= 400000, lamda = LAMBDA, check_thickness=False)
name = points.split('/')[1].split('.')[0]+f'_{a}_{b}F'
np.savetxt(points.split('.')[0]+f'_{a}_{b}F'+'.dat', af, header=points.split('/')[1].split('.')[0]+f'_{a}_{b}F', comments="")
print(f'R: {R}, angle: {a}, pose: {b}, perf: {perf}, cruise cd: {cd}, launch cd: {CD_BL}')
show_airfoil(af)
print(f'intersection: {detect_intersect(af)}, thickness: {cal_thickness(af)}, thickness pose: {cal_thickness_percent(af)}, tail cross pose: {check_backpoint(af)}')

In [ ]:
perffile = 'results/Airfoils1D_simperf.log'
f = open(perffile, 'r')
p = re.compile('perf: ')
perfs = []
airfoils = []
Rs = []
for line in f:
    if p.search(line):
        perf = line.split('perf: ')[1].split(', R: ')[0]
        perf = float(perf)
        perfs.append(perf)
        R = line.split(', R: ')[1].split(', m: ')[0]
        R = float(R)
        Rs.append(R)
        airfoilfile = line.split(', path: ')[1]
        airfoils.append(airfoilfile)

perfs = np.array(perfs)
Rs = np.array(Rs)
i = np.argmax(perfs)
k = np.argmin(Rs)
# Rs = np.delete(Rs, k)
# k = np.argmin(Rs)
# Rs = np.delete(Rs, k)
# k = np.argmin(Rs)
print(airfoils[i])
print(airfoils[k], Rs[k], perfs[k])


In [ ]:
f = open(perffile, 'r')
p = re.compile(airfoils[k])
lines = []
for line in f:
    if p.search(line) is not None:
        print(line)
        pass
    else:
        lines.append(line)
f.close()
f = open(perffile, 'w')
for line in lines:
    f.write(line)
f.close()


In [ ]:
i = 49

In [ ]:
print(i)
path = '/work3/s212645/DiffusionAirfoil1DTransform/Airfoils1D/'
path += '000.npy'
afs = np.load(path)
airfoil = afs[i]
airfoil = Normalize(airfoil)
airfoil = derotate(airfoil)
# airfoil = set_thickness_pose(airfoil, 50)
lh_idx = np.argmin(airfoil[:,0])
lh_x = airfoil[lh_idx, 0]
# Get trailing head
th_x = np.minimum(airfoil[0,0], airfoil[-1,0])
# Interpolate
f_up = interp1d(airfoil[:lh_idx+1,0], airfoil[:lh_idx+1,1])
f_low = interp1d(airfoil[lh_idx:,0], airfoil[lh_idx:,1])
xx = np.linspace(lh_x, th_x, num=1000)
yy_up = f_up(xx)
yy_low = f_low(xx)
yy_up1 = yy_up[:-2]
yy_up2 = yy_up[1:-1]
i_up = (np.abs(yy_up2-yy_up1)).argmin()
af_up = np.zeros([i_up,2])
af_up[:i_up,0] = xx[1:i_up+1]
af_up[:i_up,1] = yy_up[1:i_up+1]
af_up = af_up * 0.2 / (af_up[-1,0] - af_up[0,0])
af_up[:,1] = af_up[:,1] * 2 / af_up[-1,1]

yy_up1 = yy_low[:-2]
yy_up2 = yy_low[1:-1]
i_down = (np.abs(yy_up2-yy_up1)).argmin()
af_down = np.zeros([i_down,2])
af_down[:i_down,0] = xx[1:i_down+1]
af_down[:i_down,1] = yy_low[1:i_down+1]
af_down = af_down * 0.2 / (af_down[-1,0] - af_down[0,0])
af_down[:,1] = af_down[:,1] / abs(af_down[-1,1])

fuse = np.zeros([i_down+i_up,2])
fuse[:i_up,:] = np.flip(af_up, axis=0)
fuse[i_up:,:] = af_down
fuse[:,1] = fuse[:,1] * 0.032 / cal_thickness(fuse)

up_ = np.array([[0.45, 0.006]])
down_ = np.array([[0.45, -0.008]])
fuse = np.concatenate([up_, fuse, down_], axis = 0)

# fuse = Normalize(fuse)
fuse = interpolate(fuse, 254, 3)
up_ = np.array([[1.0, -0.008]])
down_ = np.array([[1.0, -0.014]])
fuse = np.concatenate([up_, fuse, down_], axis = 0)
fuse = Normalize(fuse)
fuse = derotate(fuse)

show_airfoil(fuse)
i+=1

In [ ]:
print(evalpreset(fuse))

In [ ]:
np.savetxt(f'F3K_airfoils/fuse.dat', fuse, header=f'fuse', comments="")

In [ ]:
i = 4
print(i)
path = '/work3/s212645/DiffusionAirfoil1DTransform/Airfoils1D/'
path += '000.npy'
afs = np.load(path)
airfoil = afs[i]
airfoil = Normalize(airfoil)
airfoil = derotate(airfoil)
# airfoil = set_thickness_pose(airfoil, 50)
lh_idx = np.argmin(airfoil[:,0])
lh_x = airfoil[lh_idx, 0]
# Get trailing head
th_x = np.minimum(airfoil[0,0], airfoil[-1,0])
# Interpolate
f_up = interp1d(airfoil[:lh_idx+1,0], airfoil[:lh_idx+1,1])
f_low = interp1d(airfoil[lh_idx:,0], airfoil[lh_idx:,1])
xx = np.linspace(lh_x, th_x, num=1000)
yy_up = f_up(xx)
yy_low = f_low(xx)
yy_up1 = yy_up[:-2]
yy_up2 = yy_up[1:-1]
i_up = (np.abs(yy_up2-yy_up1)).argmin()
af_up = np.zeros([i_up,2])
af_up[:i_up,0] = xx[1:i_up+1]
af_up[:i_up,1] = yy_up[1:i_up+1]
af_up[:,0] = af_up[:,0] ** 1.4
af_up = af_up * 0.2 / (af_up[-1,0] - af_up[0,0])
# af_up[:200,0] = af_up[:200,0] * 0.5
# af_up[200:,0] = (af_up[200:,0] -  af_up[200,0]) * 1.25 + af_up[199,0]
# af_up[:,1] = af_up[:,1] * 2 / af_up[-1,1]


af_down = np.copy(af_up)
af_down[:,1] = - af_down[:,1]

fuse = np.zeros([2*i_up,2])
fuse[:i_up,:] = np.flip(af_up, axis=0)
fuse[i_up:,:] = af_down
fuse[:,1] = fuse[:,1] * 0.04 / cal_thickness(fuse)

up_ = np.array([[0.45, 0.008], [0.4, 0.011], [0.3, 0.018]])
down_ = np.array([[0.3, -0.018], [0.4, -0.011], [0.45, -0.008]])
fuse = np.concatenate([up_, fuse, down_], axis = 0)

# # fuse = Normalize(fuse)
fuse = interpolate(fuse, 254, 3)
# up_ = np.array([[1.0, 0.004]])
# down_ = np.array([[1.0, -0.004]])
# fuse = np.concatenate([up_, fuse, down_], axis = 0)
# fuse = Normalize(fuse)
# fuse = derotate(fuse)

show_airfoil(fuse)
i+=1

In [ ]:
np.savetxt(f'F3K_airfoils/fuse_side.dat', fuse, header=f'fuse_side', comments="")

In [ ]:

points = 'samples/Airfoils2D_049.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af)
perf, _, _ = evalperf(setflap(af, 3, 0.6))
cd, cal_thickness(af), cal_thickness_percent(af), perf, check_backpoint(af)

In [ ]:

f = open('results/Airfoils1D_simperf.log', 'r')
a = f.read()
f.close()
files = []
for name in a.split('path: ')[1:]:
    file = name.split('.dat')[0] + '.dat'
    files.append(file)

In [ ]:
for file in files:
    # points = 'samples/Airfoils1D_039.dat'
    # airfoil = np.loadtxt(points, skiprows=1)
    airfoil = np.loadtxt(file, skiprows=1)
    af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, lamda = LAMBDA, check_thickness=False)
    name = file.split('/')[1].split('.')[0]+f'_{a}_{b}F'
    np.savetxt(file.split('.')[0]+f'_{a}_{b}F'+'.dat', af, header=file.split('/')[1].split('.')[0]+f'_{a}_{b}F', comments="")
    print(R, a, b, perf, cd, CD_BL)

In [ ]:
b = np.linspace(0.5, 0.6, num=5) 
a = np.linspace(-3,-1, num=5)
cdbl = 1
for pose in b:
    for alfa in a:
        afp = setupflap(airfoil, theta = alfa, pose = pose)
        cd, aa = evalpreset(afp)
        if cd < cdbl:
            pose_bl = pose
            alfa_bl = alfa
            af_bl = afp
            cdbl = cd
            print(pose_bl, alfa_bl, cdbl)

pose_bl, alfa_bl, cdbl

In [ ]:
points = 'samples/tipAirfoils1D_004.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils1D_018.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils2D_033.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils2D_044.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, -3, 0.6)
show_airfoil(af)
cd, _ = evalpreset(af, Re=1.4e5)
cd, cal_thickness(af), cal_thickness_percent(af), detect_intersect(af), check_backpoint(af)

In [ ]:
points = 'samples/tipAirfoils1D_004.dat'
airfoil = np.loadtxt(points, skiprows=1)
airfoil = setupflap(airfoil, -3, 0.6)
def set_thickness_pose(airfoil, pose):
    lh_idx = np.argmin(airfoil[:,0])
    lh_x = airfoil[lh_idx, 0]
    # Get trailing head
    th_x = np.minimum(airfoil[0,0], airfoil[-1,0])
    # Interpolate
    f_up = interp1d(airfoil[:lh_idx+1,0], airfoil[:lh_idx+1,1])
    f_low = interp1d(airfoil[lh_idx:,0], airfoil[lh_idx:,1])
    xx = np.linspace(lh_x, th_x, num=1000)
    yy_up = f_up(xx)
    yy_low = f_low(xx)
    max_thickness_index = (yy_up-yy_low).argmax()
    p = pose*10/max_thickness_index
    xxx = np.copy(xx)
    xxx[:max_thickness_index] = xx[:max_thickness_index] * p
    xxx[max_thickness_index:] = (xx[max_thickness_index:] - xx[max_thickness_index]) * (1000 - max_thickness_index * p) / (1000 - max_thickness_index) + xx[max_thickness_index] * p
    f_up = interp1d(xxx, yy_up)
    f_low = interp1d(xxx, yy_low)
    xx = np.linspace(xxx[0], xxx[-1], num=128)
    yy_up = f_up(xx)
    yy_low = f_low(xx)
    airfoil[:128,0] = np.flip(xx)
    airfoil[:128,1] = np.flip(yy_up)
    airfoil[128:,0] = xx
    airfoil[128:,1] = yy_low
    airfoil = np.delete(airfoil, [128], axis=0)
    xhat, yhat = savgol_filter((airfoil[:,0], airfoil[:,1]), 7, 3)
    airfoil[:,0] = xhat
    airfoil[:,1] = yhat
    airfoil = interpolate(airfoil, 256, 3)
    airfoil = derotate(airfoil)
    airfoil = Normalize(airfoil)
    airfoil = interpolate(airfoil, 256, 3)
    return airfoil
airfoil = set_thickness_pose(airfoil, cal_thickness_percent(airfoil)*(1-(25-23)/25))
show_airfoil(airfoil)
cal_thickness_percent(airfoil)


In [ ]:
import os

def rewrite_polar(root):
    for path, dir, files in os.walk(root):
        for file in files:
            lines=[]
            infile = root+file
            with open(infile, 'r') as fin:
                for line in fin:
                    line = line.replace('9.000  9.000', '9.000')
                    lines.append(line)
            fin.close()
            with open(infile, "w") as fout:
                for line in lines:
                    fout.write(line)
            fout.close()

root = 'Airfoils2D_049_0.6F_20_-3/'
rewrite_polar(root)

In [1]:
name = 'Airfoils1D'
airfoilpath = '/work3/s212645/DiffusionAirfoil/'+name+'/'

from utils import *
v = 5.7
mass = 0.32
area = 0.2254
d = 0.155
thickness = 0.058
reynolds = reynolds_pipe(velocity=v, diameter=d)
_cl = mode_cl(velocity=v, area=area, mass = mass)
af = np.loadtxt('BETTER/20150114-50 +2 d.dat', skiprows=1)
af = interpolate(af, 256, 3)
vs = np.linspace(5.5, 6, 3)
mass = 0.32
area = 0.2254
d = 0.155
thickness = 0.058
perf_bl = type2_simu(af, mass, d, area)
print(perf_bl)

simlog = f'results/{name}_{int(reynolds)}_{_cl:.2f}_simlog.txt'
simperf = f'results/{name}_{int(reynolds)}_{_cl:.2f}_simperf.log'
try:
    lines = []
    with open(simperf, 'r') as fin:
        for line in fin:
            lines.append(line)
    perf_bl = float(lines[-1].split('perf: ')[-1].split(', ')[0])
    print(perf_bl)
except:
    pass

try:
    log = np.loadtxt(simlog)
    i = int(log[0])
    k = int(log[1])
    m = int(log[2])
except:
    m = 0
    i = 0
    k = 0


print(f'i: {i}, k: {k}, m: {m}')
while 1:
    num = str(i).zfill(3)
    airfoils = np.load(airfoilpath+num+'.npy')
    airfoils = delete_intersect(airfoils)
    while k < airfoils.shape[0]:
        airfoil = airfoils[k,:,:]
        airfoil = derotate(airfoil)
        airfoil = Normalize(airfoil)
        airfoil = interpolate(airfoil, 400, 3)
        xhat, yhat = savgol_filter((airfoil[:,0], airfoil[:,1]), 10, 3)
        airfoil[:,0] = xhat
        airfoil[:,1] = yhat
        airfoil[:,1] = airfoil[:,1] * thickness / cal_thickness(airfoil)
        airfoil = interpolate(airfoil, 300, 3)
        if detect_intersect(airfoil):
            perf = np.nan
        else:
            try:
                perf = type2_simu(airfoil, mass, d, area)
                successful = True
                if not np.isnan(perf):
                    print(f'perf: {perf:4f}')
            except Exception as e:
                perf = np.nan
                print(e)
        if np.isnan(perf):
            pass
        elif perf > perf_bl:
            perf_bl = perf
            mm = str(m).zfill(3)
            af_name = f'{name}_{int(reynolds)}_{_cl:.2f}_{int(mm)}'
            np.savetxt(f'samplesP5B/{af_name}.dat', airfoil, header=f'{af_name}', comments="")
            f = open(simperf, 'a')
            f.write(f'perf: {perf}, m: {mm}, path: samplesP5B/{af_name}.dat\n')
            f.close()
            m += 1
            del f
        k += 1
        log = np.array([i, k, m])
        np.savetxt(simlog, log)
        del airfoil
        gc.collect()
    k = 0
    i += 1
    del airfoils
    gc.collect()

100%|██████████| 6/6 [00:01<00:00,  3.53it/s]


39.948841364498186
48.16769420378617
i: 4, k: 725, m: 2


 33%|███▎      | 2/6 [00:00<00:01,  2.06it/s]

In [ ]:
from utils import *
v = 5.3
mass = 0.32
area = 0.22
d = 0.05
thickness = 0.048
reynolds = reynolds_pipe(velocity=v, diameter=d)
cl = mode_cl(velocity=v, area=area, mass = mass)
af = np.loadtxt('BETTER/20150114-50 +2 d.dat', skiprows=1)
af = interpolate(af, 256, 3)
perf_bl, _, _ = evalperf(af, cl=cl, Re=reynolds)
print(perf_bl)
perf_bl = 16.446707872028764
cd_bl = 0.023

name = 'Airfoils1D'
airfoilpath = '/work3/s212645/DiffusionAirfoil/'+name+'/'
simlog = f'results/{name}_{int(reynolds)}_{cl:.2f}_simlog.txt'
simperf = f'results/{name}_{int(reynolds)}_{cl:.2f}_simperf.log'

try:
    log = np.loadtxt(simlog)
    i = int(log[0])
    k = int(log[1])
    m = int(log[2])
except:
    m = 0
    i = 0
    k = 0


print(f'i: {i}, k: {k}, m: {m}')
while 1:
    num = str(i).zfill(3)
    airfoils = np.load(airfoilpath+num+'.npy')
    airfoils = delete_intersect(airfoils)
    while k < airfoils.shape[0]:
        airfoil = airfoils[k,:,:]
        airfoil = derotate(airfoil)
        airfoil = Normalize(airfoil)
        airfoil = interpolate(airfoil, 256, 3)
        xhat, yhat = savgol_filter((airfoil[:,0], airfoil[:,1]), 10, 3)
        airfoil[:,0] = xhat
        airfoil[:,1] = yhat
        airfoil[:,1] = airfoil[:,1] * thickness / cal_thickness(airfoil)
        airfoil = interpolate(airfoil, 256, 3)
        if cal_camber(airfoil) > 9:
            airfoil = set_camber(airfoil)
        xf = XFoil()
        xf.print = 0
        xf.airfoil = Airfoil(airfoil[:,0], airfoil[:,1])
        xf.Re = reynolds_pipe(velocity=v, diameter=d)
        xf.max_iter = 1000
        a, cl, cd, cm, cp = xf.aseq(0,4,0.5)
        del xf
        gc.collect()
        i_nan = np.argwhere(np.isnan(cd))
        cd = np.delete(cd, i_nan)
        cl = np.delete(cl, i_nan)
        a = np.delete(a, i_nan)
        perf = cl / cd
        try:
            perf = perf.max()
            print(perf)
        except:
            perf = np.nan
        # xf = XFoil()
        # xf.print = 0
        # xf.airfoil = Airfoil(airfoil[:,0], airfoil[:,1])
        # xf.Re = reynolds
        # xf.max_iter = 1000
        # a, cl, cd, cm, cp = xf.aseq(0,5,0.5)
        # del xf
        # gc.collect()
        # i_nan = np.argwhere(np.isnan(cl))
        # a = np.delete(a, i_nan)
        # cd = np.delete(cd, i_nan)
        # cl = np.delete(cl, i_nan)
        # try:
        #     cd = cd.min()
        #     print(cd)
        # except:
        #     cd = np.nan
        # successful = False
        # while not successful:
        #     try:
        #         perf, _, _ = evalperf(airfoil, cl=0.42, Re=reynolds)
        #         successful = True
        #         if not np.isnan(perf):
        #             print(f'perf: {perf:4f}')
        #     except Exception as e:
        #         print(e)
        #         break
        if perf == np.nan:
            pass
        elif perf > perf_bl and cal_thickness_percent(airfoil) < 25 and cal_camber(airfoil) < 9 and cal_camber(airfoil) > 0:
            mm = str(m).zfill(3)
            af_name = f'{name}_{int(reynolds)}_{int(mm)}_tip'
            np.savetxt(f'samplesP5B/{af_name}.dat', airfoil, header=f'{af_name}', comments="")
            f = open(simperf, 'a')
            f.write(f'perf: {perf}, m: {mm}, path: samplesP5B/{af_name}.dat\n')
            f.close()
            m += 1
            del f
        k += 1
        log = np.array([i, k, m])
        np.savetxt(simlog, log)
        del airfoil
        gc.collect()
    k = 0
    i += 1
    del airfoils
    gc.collect()

In [ ]:
af = np.loadtxt('samplesP5B/bo_refine_2.dat', skiprows=1)
show_airfoil(af)

v = 5.7
mass = 0.32
area = 0.2254
d = 0.155
thickness = 0.058
reynolds = reynolds_pipe(velocity=v, diameter=d)
xf = XFoil()
xf.print = 0
xf.airfoil = Airfoil(af[:,0], af[:,1])
xf.Re = reynolds_pipe(velocity=v, diameter=d)
xf.max_iter = 1000
a, cl, cd, cm, cp = xf.aseq(0,5,0.5)
del xf
gc.collect()
i_nan = np.argwhere(np.isnan(cd))
cd = np.delete(cd, i_nan)
cl = np.delete(cl, i_nan)
a = np.delete(a, i_nan)
perf = cl / cd
i = perf.argmax()
perf.max(), cl[i], cal_thickness_percent(af), check_backpoint(af), cal_camber(af)

In [ ]:
af = np.loadtxt('samplesP5B/Airfoils1D_18034_25_tip.dat', skiprows=1)
show_airfoil(af)
cal_thickness_percent(af)

In [ ]:
af = np.loadtxt('samplesP5B/DiffusionAirfoil1DTransform_60127_0.70_3.dat', skiprows=1)
show_airfoil(af)

v = 5.7
mass = 0.32
area = 0.2254
d = 0.155
thickness = 0.058
reynolds = reynolds_pipe(velocity=v, diameter=d)
cl = mode_cl(velocity=v, area=area, mass = mass)
af = interpolate(af, 256, 3)
perf, a, _ = evalperf(af, cl=cl, Re=reynolds)
perf, a